# DATA CLEANING PROCEDURES

---
# 1. Loading the Database

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')
columns_description = pd.read_csv('column_descriptions.csv')

# Quick overview
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("\nTrain columns and types:\n", train.dtypes.head())
train.head() # Display first few rows of the train dataset

---
# 2. Checking missing values

We define a function which summarizes:
* how many missing values each column has
* what % of data is missing
* only columns with missing values are shown
* results are sorted from worst to best

We check both Train and Test datasets, and merge the results for easy comparison.

In [ ]:
# Function to summarize missing values
def missing_summary(df, dataset_name="Dataset"):
    total = df.isnull().sum()
    percent = (total / len(df)) * 100
    missing_table = pd.concat([total, percent], axis=1)
    missing_table.columns = [f'Missing Values ({dataset_name})', f'% Missing ({dataset_name})']
    missing_table = missing_table[missing_table[f'Missing Values ({dataset_name})'] > 0]
    return missing_table.sort_values(f'% Missing ({dataset_name})', ascending=False)

# Summarizing missing values for train and test
missing_train = missing_summary(train, "Train")
missing_test = missing_summary(test, "Test")

# Merging both summaries together
missing_values = missing_train.merge(missing_test, how='outer', left_index=True, right_index=True)

# Sort by maximum % missing across Train and Test
missing_values['Max % Missing'] = missing_values[
    [col for col in missing_values.columns if '% Missing' in col]
].max(axis=1)
missing_values = missing_values.sort_values(by='Max % Missing', ascending=False)
missing_values.drop(columns='Max % Missing', inplace=True)

# Display the missing values table
print(missing_values)

# Visualizing missing values
plt.figure(figsize=(12, 6))
missing_values[
    [col for col in missing_values.columns if '% Missing' in col]
].plot(kind='bar', figsize=(14, 6))
plt.title("Missing Values (%): Train vs Test")
plt.ylabel("Percentage Missing")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

---
# 3. Planning the Cleaning Strategy

After analyzing the missing values, we decided cleaning actions based on two criteria:
- percentage of missing values;
- importance of each column for predicting player market value

Actions applied:
* DROP columns with too many missing values (>50%) -> if they are not critical 
* FILL missing values in numerical columns -> using median to avoid influence from outliers
* FILL missing values in categorical columns -> with 'unknown'

In [ ]:
# Columns to drop
columns_to_drop = [
    'player_traits', 'player_tags', 'club_loaned_from', 'nation_jersey_number', 
    'nation_position', 'goal_keeping_speed']

# Numerical Columns to Fill with Median
numerical_cols = [
    'release_clause_eur', 'defending', 'dribbling', 'pace',
    'passing', 'physic', 'shooting', 'wage_eur']
# (Exclude club_jersey_number: not relevant)

# Categorical Columns to Fill with 'Unknown'
categorical_cols = [
    'club_name', 'club_position', 'league_name', 'club_contract_valid_until']

---
# 4. Applying the Cleaning

In [ ]:
# 1. DROP columns -> too many missing values + not critical for predicting market value
train.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test.drop(columns=columns_to_drop, inplace=True, errors='ignore')
# Quick check : dropped columns
print("Dropped columns from train:", columns_to_drop)
print("Dropped columns from test:", columns_to_drop)

# 2. FILL numerical columns -> with median
for column in numerical_cols: 
    median_value = train[column].median()
    train[column] = train[column].fillna(median_value)
    test[column] = test[column].fillna(median_value)
# Quick check: filled numerical columns
print("Train numerical columns after filling:", train[numerical_cols].isnull().sum())
print("Test numerical columns after filling:", test[numerical_cols].isnull().sum())

# 3. FILL categorical columns -> with 'Unknown'
for column in categorical_cols:
    train[column] = train[column].fillna('Unknown')
    test[column] = test[column].fillna('Unknown')   
# Quick check: filled categorical columns
print("Train categorical columns after filling:", train[categorical_cols].isnull().sum())
print("Test categorical columns after filling:", test[categorical_cols].isnull().sum())

#4. FILL remaining missing values -> with 'Unknown'
train.fillna('Unknown', inplace=True)
test.fillna('Unknown', inplace=True)
# Last check: filled remaining missing values
print("Remaining missing values in train:", train.isnull().sum().sum())
print("Remaining missing values in test:", test.isnull().sum().sum())


---
# 5. Checking database Consistency

Before training a model, we need to make sure that the train and test datasets have the same structure.

We need to take into account that the train set contains the target column 'value_eur', while the test set does not because we still need to predict it. So we temporarily drop this column from the train set before comparing

In [ ]:
train_features = train.drop(columns=['value_eur'])

# Checking if train and test datasets have the same columns
train_columns = set(train_features.columns)
test_columns = set(test.columns)

if train_columns != test_columns:
    print("Mismatch in columns between train and test datasets:")
    print("Columns in train but not in test:", train_columns - test_columns)
    print("Columns in test but not in train:", test_columns - train_columns)
else:
    print("Train and test datasets have the same columns.")


---
# 6. Final Checks


In [ ]:
# Checking for unexpected missing values
missing_train = train.isnull().sum()[train.isnull().sum() > 0]
if missing_train.empty:
    print("No unexpected missing values in train dataset. ")
else:
    print(missing_train)

missing_test = test.isnull().sum()[test.isnull().sum() > 0]
if missing_test.empty:
    print("No unexpected missing values in test dataset. ")
else:
    print(missing_test)

---
# 7. Conclusion
This data cleaning process helped us to make sure that the datasets are consistent and ready for modelling. 

We addressed missing values by dropping irrelevant columns, properly filling missing values , and verifying that the train and test datasets have matching structures. 

With the data now cleaned and organized, we're ready to build and evaluate predictive models.